In [16]:
import pandas as pd
import numpy as np
import re
from scipy import sparse
from sklearn import preprocessing, linear_model
import sklearn.feature_extraction.text as txt


train = pd.read_csv('TrainingData.csv')
test = pd.read_csv('TestData.csv')
sample = pd.read_csv('SubmissionFormat.csv')

training_data = train[['FTE','Facility_or_Department', 'Function_Description','Fund_Description',
                       'Job_Title_Description', 'Location_Description','Object_Description',
                       'Position_Extra', 'Program_Description', 'SubFund_Description',
                       'Sub_Object_Description', 'Text_1', 'Text_2','Text_3','Text_4', 'Total']]

cols = ['Function','Object_Type','Operating_Status','Position_Type','Pre_K', 'Reporting',
                'Sharing','Student_Type', 'Use']

labels = train[cols]


In [18]:

it=['Facility_or_Department', 'Function_Description','Fund_Description',
                       'Job_Title_Description', 'Location_Description','Object_Description',
                       'Position_Extra', 'Program_Description', 'SubFund_Description',
                       'Sub_Object_Description', 'Text_1', 'Text_2','Text_3','Text_4']
test_data = test[['FTE','Facility_or_Department', 'Function_Description','Fund_Description',
                       'Job_Title_Description', 'Location_Description','Object_Description',
                       'Position_Extra', 'Program_Description', 'SubFund_Description',
                       'Sub_Object_Description', 'Text_1', 'Text_2','Text_3','Text_4', 'Total']]


label_encoder = preprocessing.LabelEncoder()
for i in cols:
    labels[i] = label_encoder.fit_transform(np.array(labels[i]))
    
    
training_data = training_data.drop('FTE', axis = 1)
training_data = training_data.drop('Total', axis = 1)

test_data = test_data.drop('FTE', axis = 1)
test_data = test_data.drop('Total', axis = 1)


def clean(s):
    try:
        return " ".join(re.findall(r'\w+', s)).lower()
    except:
        return " ".join(re.findall(r'\w+', "no_text")).lower()

for i in it:
        training_data[i] = training_data[i].apply(clean)
for i in it:
        test_data[i] = test_data[i].apply(clean)
        


C:\Users\derri\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [19]:
training_data["combined"] = [' '.join(row) for row in training_data[training_data.columns].values]
test_data["combined"] = [' '.join(row) for row in test_data[test_data.columns].values]


In [22]:
tfidf = txt.TfidfVectorizer(ngram_range=(1,2), max_df=1.0, min_df=10)
hsv = txt.HashingVectorizer()


tfidf.fit(training_data['combined'])
hsv.fit(test_data['combined'])

X_tfidf = tfidf.transform(training_data['combined'])
X_test_tfidf = tfidf.transform(test_data['combined'])

X_hsv = hsv.transform(training_data['combined'])
X_test_hsv = hsv.transform(test_data['combined'])

X = sparse.hstack((X_hsv, X_tfidf))
X_test = sparse.hstack((X_test_hsv, X_test_tfidf))


In [23]:
prediction1 = []
prediction2 = []
prediction3 = []
prediction4 = []
for i in range(len(cols)):
    print ("label = ", i)
    sgd1 = linear_model.SGDClassifier(loss = 'log', max_iter = 120, alpha = 0.000001)
    sgd2 = linear_model.SGDClassifier(loss = 'log', max_iter = 120, penalty = 'l1')
    sgd3 = linear_model.SGDClassifier(loss = 'log', max_iter = 120, penalty = 'l2')
    sgd4 = linear_model.SGDClassifier(loss = 'log', max_iter = 120, penalty = 'elasticnet')
    sgd1.fit(X, labels.iloc[:,i])
    sgd2.fit(X, labels.iloc[:,i])
    sgd3.fit(X, labels.iloc[:,i])
    sgd4.fit(X, labels.iloc[:,i])
    if i == 0:
        prediction1 = sgd1.predict_proba(X_test)
        prediction2 = sgd2.predict_proba(X_test)
        prediction3 = sgd3.predict_proba(X_test)
        prediction4 = sgd4.predict_proba(X_test)
    else:
        print(len(prediction1),len(sgd1.predict_proba(X_test)))
        prediction1 = np.hstack((prediction1,sgd1.predict_proba(X_test)))
        prediction2 = np.hstack((prediction2,sgd2.predict_proba(X_test)))
        prediction3 = np.hstack((prediction3,sgd3.predict_proba(X_test)))
        prediction4 = np.hstack((prediction4,sgd4.predict_proba(X_test)))
        
preds = (prediction1 + prediction2 + prediction3 + prediction4)/4.0

for i in range(1, len(sample.columns)):
    colname = sample.columns[i]
    sample[str(colname)] = preds[:,i-1]

sample.to_csv('Logy_Model.csv', index = False)

label =  0
label =  1
50064 50064
label =  2
50064 50064
label =  3
50064 50064
label =  4
50064 50064
label =  5
50064 50064
label =  6
50064 50064
label =  7
50064 50064
label =  8
50064 50064
